In [7]:
from sae_lens.toolkit.pretrained_saes import load_sparsity
import plotly.express as px

path = "open_ai_sae_weights_resid_post_attn_reformatted/v5_32k_layer_0"


sparsity = load_sparsity(path)  # ["sparsity"]


px.histogram(sparsity.cpu(), nbins=100).write_html("sparsity_histogram.html")

In [57]:
import pandas as pd

# get all json files in all subfolders of the mother path
import os
import json
from IPython.display import display
import imgkit


def get_all_json_files(mother_path):
    json_files = []

    for root, dirs, files in os.walk(mother_path):
        for file in files:
            if file.endswith("metrics.json"):
                json_files.append(os.path.join(root, file))
    return json_files


def get_benchmark_stats_csv(
    mother_path="open_ai_sae_weights_resid_post_attn_reformatted",
):
    json_files = get_all_json_files(mother_path)
    eval_metrics = {}

    for file in json_files:
        with open(file, "r") as f:
            data = json.load(f)
            eval_metrics[file] = data

    df = pd.DataFrame(eval_metrics).T
    df["filepath"] = df.index
    df.head()
    pattern = r".*/v(\d+)_(\d+)k_layer_(\d+)/metrics\.json"

    df[["version", "d_sae", "layer"]] = df.filepath.str.extract(pattern)
    # move these columns to the start
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3]
    df = df[cols]
    df["layer"] = df["layer"].astype(int)

    # remove "metrics" prefix from the columns
    df.columns = [i.replace("metrics/", "") for i in df.columns]
    df.sort_values(by=["version", "d_sae", "layer"], inplace=True)
    df.to_csv(os.path.join(mother_path, "benchmark_stats.csv"))
    df.style.background_gradient(cmap="viridis", axis=0).to_html(
        os.path.join(mother_path, "benchmark_stats.html")
    )

    # read the html
    with open(os.path.join(mother_path, "benchmark_stats.html"), "r") as f:
        html = f.read()
        imgkit.from_string(html, os.path.join(mother_path, "benchmark_stats.png"))

    return df.style.background_gradient(cmap="viridis", axis=0)


# list all paths that start with OAI in the current fold
paths = [i for i in os.listdir(".") if i.startswith("OAI")]
tables = []
for path in paths:
    print(path)
    display(get_benchmark_stats_csv(path))

OAI_GPT2Small_v5_32k_resid_delta_mlp
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


,version,d_sae,layer,kl_div_with_sae,kl_div_with_ablation,ce_loss_with_sae,ce_loss_without_sae,ce_loss_with_ablation,kl_div_score,ce_loss_score,l2_norm_in,l2_norm_out,l2_ratio,l0,l1,explained_variance,mse,total_tokens_evaluated,filepath
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_0/metrics.json,5,32,0,0.004845,3.094083,3.605465,3.599065,6.694649,0.998434,0.997933,29.933449,29.601543,0.989371,32.000000,71.211151,0.966797,21.729292,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_0/metrics.json
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_1/metrics.json,5,32,1,0.006601,0.051053,3.605596,3.599065,3.652537,0.870694,0.877862,18.973736,17.917168,0.910649,32.000000,86.565331,0.885442,25.637442,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_1/metrics.json
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_2/metrics.json,5,32,2,0.009369,0.058747,3.601879,3.599065,3.645913,0.840524,0.939922,49.106537,47.644482,0.888798,31.875000,85.811630,0.974547,37.837296,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_2/metrics.json
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_3/metrics.json,5,32,3,0.010681,0.070592,3.609601,3.599065,3.658678,0.848690,0.823245,16.987318,15.157210,0.874669,31.911459,85.938217,0.780534,50.548058,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_3/metrics.json
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_4/metrics.json,5,32,4,0.012658,0.063325,3.611159,3.599065,3.660080,0.800111,0.801781,17.251986,15.012179,0.852544,31.955566,82.476707,0.729496,63.704514,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_4/metrics.json
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_5/metrics.json,5,32,5,0.014467,0.068505,3.613976,3.599065,3.669386,0.788825,0.787950,18.888968,16.209919,0.848440,32.000000,81.434013,0.717422,87.281723,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_5/metrics.json
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_6/metrics.json,5,32,6,0.016600,0.075694,3.618799,3.599065,3.676516,0.780703,0.745207,21.466564,18.402473,0.852635,32.000000,78.829765,0.706308,117.072495,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_6/metrics.json
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_7/metrics.json,5,32,7,0.017010,0.080486,3.614976,3.599065,3.672712,0.788663,0.783952,25.444439,22.004990,0.862489,32.000000,76.419937,0.718003,157.791412,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_7/metrics.json
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_8/metrics.json,5,32,8,0.018103,0.087324,3.616245,3.599065,3.680337,0.792688,0.788606,30.250225,26.306936,0.867637,32.000000,76.728195,0.723916,219.982910,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_8/metrics.json
OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_9/metrics.json,5,32,9,0.019997,0.097589,3.617456,3.599065,3.696245,0.795088,0.810751,40.192413,35.945808,0.889800,32.000000,72.426567,0.742352,318.143433,6144.000000,OAI_GPT2Small_v5_32k_resid_delta_mlp/v5_32k_layer_9/metrics.json


OAI_GPT2Small_v5_128k_resid_delta_mlp
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


,version,d_sae,layer,kl_div_with_sae,kl_div_with_ablation,ce_loss_with_sae,ce_loss_without_sae,ce_loss_with_ablation,kl_div_score,ce_loss_score,l2_norm_in,l2_norm_out,l2_ratio,l0,l1,explained_variance,mse,total_tokens_evaluated,filepath
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_0/metrics.json,5,128,0,0.002883,3.094083,3.601894,3.599065,6.694649,0.999068,0.999086,29.933449,29.719198,0.993006,31.999350,61.198639,0.977519,13.276111,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_0/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_1/metrics.json,5,128,1,0.004242,0.051053,3.599821,3.599065,3.652537,0.916919,0.985857,18.973736,18.205364,0.932765,32.000000,84.060181,0.916917,17.682375,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_1/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_2/metrics.json,5,128,2,0.006752,0.058747,3.602034,3.599065,3.645913,0.885070,0.936614,49.106537,47.976685,0.912467,31.984375,82.676140,0.981090,28.213791,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_2/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_3/metrics.json,5,128,3,0.008532,0.070592,3.607723,3.599065,3.658678,0.879137,0.854756,16.987318,15.537837,0.899518,31.905111,81.886444,0.827494,38.814980,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_3/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_4/metrics.json,5,128,4,0.010456,0.063325,3.611892,3.599065,3.660080,0.834885,0.789762,17.251986,15.434065,0.879446,31.984375,78.371658,0.780778,50.818203,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_4/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_5/metrics.json,5,128,5,0.011618,0.068505,3.609259,3.599065,3.669386,0.830411,0.855026,18.888968,16.669851,0.873767,31.983725,77.139160,0.765372,71.502914,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_5/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_6/metrics.json,5,128,6,0.014023,0.075694,3.614241,3.599065,3.676516,0.814737,0.804058,21.466564,18.890602,0.875795,31.998373,74.617645,0.754052,97.221252,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_6/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_7/metrics.json,5,128,7,0.014005,0.080486,3.609832,3.599065,3.672712,0.825992,0.853803,25.444439,22.559195,0.884380,31.997885,72.551300,0.764806,130.780823,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_7/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_8/metrics.json,5,128,8,0.014951,0.087324,3.615898,3.599065,3.680337,0.828790,0.792874,30.250225,26.934555,0.888432,31.996908,72.559128,0.768030,183.634979,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_8/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_9/metrics.json,5,128,9,0.015919,0.097589,3.615844,3.599065,3.696245,0.836876,0.827333,40.192413,36.576889,0.905792,32.000000,68.976295,0.782096,267.216095,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_mlp/v5_128k_layer_9/metrics.json


OAI_GPT2Small_v5_32k_resid_delta_attn
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


...

,version,d_sae,layer,kl_div_with_sae,kl_div_with_ablation,ce_loss_with_sae,ce_loss_without_sae,ce_loss_with_ablation,kl_div_score,ce_loss_score,l2_norm_in,l2_norm_out,l2_ratio,l0,l1,explained_variance,mse,total_tokens_evaluated,filepath
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_0/metrics.json,5,128,0,0.003102,2.121528,3.600031,3.599065,5.748601,0.998538,0.999551,32.013138,31.910549,0.996805,31.989422,39.967445,0.973218,6.201825,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_0/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_1/metrics.json,5,128,1,0.001700,0.024066,3.601379,3.599065,3.620633,0.929377,0.892674,9.714648,9.198002,0.942320,31.999023,78.196198,0.879798,8.594531,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_1/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_2/metrics.json,5,128,2,0.002000,0.031005,3.599786,3.599065,3.626660,0.935502,0.973847,8.641823,8.086132,0.934166,31.987143,79.245674,0.869267,8.591892,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_2/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_3/metrics.json,5,128,3,0.002505,0.025134,3.601374,3.599065,3.628661,0.900320,0.921957,8.571012,7.854372,0.916164,32.000000,77.344894,0.837647,11.842029,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_3/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_4/metrics.json,5,128,4,0.003484,0.026723,3.601578,3.599065,3.632133,0.869623,0.924002,9.123016,8.117954,0.891417,31.999350,75.460785,0.798166,17.370716,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_4/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_5/metrics.json,5,128,5,0.003556,0.031378,3.601702,3.599065,3.627760,0.886681,0.908081,10.034396,8.960555,0.894489,31.998373,73.329163,0.804148,22.216118,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_5/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_6/metrics.json,5,128,6,0.004436,0.032789,3.603125,3.599065,3.634286,0.864723,0.884709,11.678066,10.348961,0.888121,31.998699,70.520447,0.776109,31.851517,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_6/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_7/metrics.json,5,128,7,0.004356,0.034661,3.602255,3.599065,3.634834,0.874331,0.910796,13.650208,12.425106,0.911562,31.999350,66.425446,0.803248,37.544487,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_7/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_8/metrics.json,5,128,8,0.005140,0.029382,3.602755,3.599065,3.625802,0.825072,0.861990,16.137949,14.539435,0.902075,32.000000,66.759209,0.776774,52.779171,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_8/metrics.json
OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_9/metrics.json,5,128,9,0.004128,0.028918,3.601983,3.599065,3.636501,0.857246,0.922041,20.912498,19.252647,0.923308,31.999350,60.197113,0.800841,70.065033,6144.000000,OAI_GPT2Small_v5_128k_resid_delta_attn/v5_128k_layer_9/metrics.json


OAI_GPT2Small_v5_32k_resid_post_attn
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


,version,d_sae,layer,kl_div_with_sae,kl_div_with_ablation,ce_loss_with_sae,ce_loss_without_sae,ce_loss_with_ablation,kl_div_score,ce_loss_score,l2_norm_in,l2_norm_out,l2_ratio,l0,l1,explained_variance,mse,total_tokens_evaluated,filepath
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_0/metrics.json,5,32,0,0.012114,12.480284,3.614269,3.599065,15.861977,0.999029,0.998760,32.707962,32.566856,0.995715,31.989258,44.145779,0.967972,8.511816,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_0/metrics.json
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_1/metrics.json,5,32,1,0.010769,16.217104,3.608070,3.599065,19.600266,0.999336,0.999437,56.929867,56.553204,0.993499,32.000000,50.736076,0.960833,47.572258,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_1/metrics.json
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_2/metrics.json,5,32,2,0.020159,12.813511,3.623466,3.599065,16.327873,0.998427,0.998083,68.907532,68.362450,0.991143,31.999350,52.513954,0.957198,71.102158,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_2/metrics.json
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_3/metrics.json,5,32,3,0.035772,10.101868,3.636966,3.599065,13.548822,0.996459,0.996191,103.711441,102.807175,0.986217,31.925781,53.469627,0.965960,125.999268,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_3/metrics.json
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_4/metrics.json,5,32,4,0.057062,13.249713,3.668521,3.599065,16.699104,0.995693,0.994698,111.403282,109.931808,0.979109,31.931152,55.739113,0.951081,211.805756,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_4/metrics.json
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_5/metrics.json,5,32,5,0.069624,11.519682,3.673068,3.599065,14.860109,0.993956,0.993428,119.651489,117.608505,0.973188,31.877768,56.817348,0.937413,317.223206,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_5/metrics.json
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_6/metrics.json,5,32,6,0.087217,6.933250,3.691099,3.599065,10.522690,0.987420,0.986707,128.847931,126.086975,0.967220,31.995281,58.897686,0.923337,455.123871,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_6/metrics.json
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_7/metrics.json,5,32,7,0.102682,9.511523,3.708053,3.599065,13.054041,0.989204,0.988473,140.905991,137.484528,0.964016,31.886557,56.659546,0.910139,636.605774,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_7/metrics.json
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_8/metrics.json,5,32,8,0.123620,7.897106,3.733898,3.599065,11.460873,0.984346,0.982850,157.343246,153.033264,0.961115,31.998699,55.846397,0.892514,920.340698,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_8/metrics.json
OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_9/metrics.json,5,32,9,0.136715,5.396312,3.744588,3.599065,8.970472,0.974665,0.972908,181.313721,175.997467,0.960342,31.993652,54.271484,0.871873,1386.967773,6144.000000,OAI_GPT2Small_v5_32k_resid_post_attn/v5_32k_layer_9/metrics.json


OAI_GPT2Small_v5_128k_resid_post_attn
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


,version,d_sae,layer,kl_div_with_sae,kl_div_with_ablation,ce_loss_with_sae,ce_loss_without_sae,ce_loss_with_ablation,kl_div_score,ce_loss_score,l2_norm_in,l2_norm_out,l2_ratio,l0,l1,explained_variance,mse,total_tokens_evaluated,filepath
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_0/metrics.json,5,128,0,0.003843,12.480284,3.603422,3.599065,15.861977,0.999692,0.999645,32.707962,32.607395,0.996936,31.980795,44.247345,0.976495,5.842685,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_0/metrics.json
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_1/metrics.json,5,128,1,0.006731,16.217104,3.605462,3.599065,19.600266,0.999585,0.999600,56.929867,56.693493,0.995869,31.999023,59.953354,0.972269,31.145605,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_1/metrics.json
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_2/metrics.json,5,128,2,0.014035,12.813511,3.612803,3.599065,16.327873,0.998905,0.998921,68.907532,68.518921,0.993602,31.999350,53.266327,0.969186,49.732628,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_2/metrics.json
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_3/metrics.json,5,128,3,0.023511,10.101868,3.622775,3.599065,13.548822,0.997673,0.997617,103.711441,103.026962,0.989538,31.983074,54.825603,0.975295,89.620079,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_3/metrics.json
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_4/metrics.json,5,128,4,0.037530,13.249713,3.640705,3.599065,16.699104,0.997167,0.996821,111.403282,110.286774,0.984007,31.948568,56.497829,0.963923,153.826294,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_4/metrics.json
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_5/metrics.json,5,128,5,0.047411,11.519682,3.644166,3.599065,14.860109,0.995884,0.995995,119.651489,118.053459,0.978980,31.958008,56.149471,0.952276,238.126266,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_5/metrics.json
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_6/metrics.json,5,128,6,0.060794,6.933250,3.655243,3.599065,10.522690,0.991232,0.991886,128.847931,126.688171,0.974231,31.996094,56.119469,0.940335,349.569763,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_6/metrics.json
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_7/metrics.json,5,128,7,0.073590,9.511523,3.668211,3.599065,13.054041,0.992263,0.992687,140.905991,138.128754,0.970831,31.997070,55.752361,0.928668,499.765717,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_7/metrics.json
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_8/metrics.json,5,128,8,0.089368,7.897106,3.679746,3.599065,11.460873,0.988683,0.989738,157.343246,153.839539,0.968352,31.998373,53.455093,0.913877,732.317871,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_8/metrics.json
OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_9/metrics.json,5,128,9,0.096923,5.396312,3.695744,3.599065,8.970472,0.982039,0.982001,181.313721,176.829346,0.966578,31.997070,51.393932,0.895696,1123.425049,6144.000000,OAI_GPT2Small_v5_128k_resid_post_attn/v5_128k_layer_9/metrics.json
